In [1]:
import csv
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
url = 'https://books.toscrape.com/catalogue/category/books_1/index.html'

In [3]:
print(url)

https://books.toscrape.com/catalogue/category/books_1/index.html


In [4]:
req = requests.get(url)

In [5]:
con = req.content

In [6]:
soup = BeautifulSoup(con , 'html.parser')

In [7]:
soup


<!DOCTYPE html>

<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-us"> <!--<![endif]-->
<head>
<title>
    Books | 
     Books to Scrape - Sandbox

</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="24th Jun 2016 09:29" name="created"/>
<meta content="
    
" name="description"/>
<meta content="width=device-width" name="viewport"/>
<meta content="NOARCHIVE,NOCACHE" name="robots"/>
<!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
<!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
<link href="../../../static/oscar/favicon.ico" rel="shortcut icon"/>
<link href="../../../static/oscar/css/styles.css" rel="stylesheet" type="

In [8]:
lis_cat = soup.find('ul' , attrs={'class' : 'nav nav-list'}).find_all('li')[1:]

In [31]:
lis_cat[0].text.strip()

'Travel'

In [11]:
site_url = 'https://books.toscrape.com/catalogue/category'

In [35]:
cats = {cat.text.strip() : site_url + cat.find('a')['href'].replace('..' , '') for cat in lis_cat}

In [37]:
cats['Travel']

'https://books.toscrape.com/catalogue/category/books/travel_2/index.html'

In [13]:
cats[0]

'https://books.toscrape.com/catalogue/category/books/travel_2/index.html'

In [57]:
def scrape(url):
    
    req = requests.get(url)
    con = req.content
    soup = BeautifulSoup(con , 'html.parser')
    
    
    section = soup.find('section')
    ol = section.find('ol')
    lis = ol.find_all('li')
    
    names , rates , price , stock , books_url  = [] , [] , [] , [] , [] 
    
    for li in lis:
    
        book_name = li.find('div' , attrs = {'class' : 'image_container'}).find('a').find('img')['alt']
        names.append(book_name)
#         print(book_name)

        book_rate = li.find('p' , attrs = {'class' : 'star-rating'})['class'][1]
        rates.append(book_rate)
#         print(book_rate)

        book_price = float(li.find('p' , attrs = {'class' : 'price_color'}).text[1:])
        price.append(book_price)
#         print(book_price)

        ava_stock = li.find('p' , attrs = {'class' : 'instock availability'}).text.strip()
        stock.append(ava_stock)
#         print(ava_stock)

        book_url = 'https://books.toscrape.com/catalogue/' + li.find('h3').find('a')['href'].replace('../' , '')
        books_url.append(book_url)
#         print(book_url)


#         print('*'*30)
    next_pg = ''   
    try:    
        next_pg = soup.find('ul' , attrs = {'class' : 'pager'}).find('li' , attrs = {'class' : 'next'}).find('a')['href']
    except:
        pass
    
    return names , rates , price , stock , books_url , next_pg

In [38]:
scrape(cats['Travel'])

It's Only the Himalayas
Two
45.17
In stock
https://books.toscrape.com/catalogue/its-only-the-himalayas_981/index.html
******************************
Full Moon over Noah’s Ark: An Odyssey to Mount Ararat and Beyond
Four
49.43
In stock
https://books.toscrape.com/catalogue/full-moon-over-noahs-ark-an-odyssey-to-mount-ararat-and-beyond_811/index.html
******************************
See America: A Celebration of Our National Parks & Treasured Sites
Three
48.87
In stock
https://books.toscrape.com/catalogue/see-america-a-celebration-of-our-national-parks-treasured-sites_732/index.html
******************************
Vagabonding: An Uncommon Guide to the Art of Long-Term World Travel
Two
36.94
In stock
https://books.toscrape.com/catalogue/vagabonding-an-uncommon-guide-to-the-art-of-long-term-world-travel_552/index.html
******************************
Under the Tuscan Sun
Three
37.33
In stock
https://books.toscrape.com/catalogue/under-the-tuscan-sun_504/index.html
******************************
A 

(["It's Only the Himalayas",
  'Full Moon over Noah’s Ark: An Odyssey to Mount Ararat and Beyond',
  'See America: A Celebration of Our National Parks & Treasured Sites',
  'Vagabonding: An Uncommon Guide to the Art of Long-Term World Travel',
  'Under the Tuscan Sun',
  'A Summer In Europe',
  'The Great Railway Bazaar',
  'A Year in Provence (Provence #1)',
  'The Road to Little Dribbling: Adventures of an American in Britain (Notes From a Small Island #2)',
  'Neither Here nor There: Travels in Europe',
  '1,000 Places to See Before You Die'],
 ['Two',
  'Four',
  'Three',
  'Two',
  'Three',
  'Two',
  'One',
  'Four',
  'One',
  'Three',
  'Five'],
 [45.17, 49.43, 48.87, 36.94, 37.33, 44.34, 30.54, 56.88, 23.21, 38.95, 26.08],
 ['In stock',
  'In stock',
  'In stock',
  'In stock',
  'In stock',
  'In stock',
  'In stock',
  'In stock',
  'In stock',
  'In stock',
  'In stock'],
 ['https://books.toscrape.com/catalogue/its-only-the-himalayas_981/index.html',
  'https://books.toscra

In [45]:
import os 

In [59]:
def StoreData(cat , names , rates , price , stock , books_url):
    
    m = 'w' if not os.path.exists('BooksData.csv') else 'a'
    
    with open('BooksData.csv' ,mode=m , newline='') as f :
    
        writer = csv.DictWriter(f , fieldnames=[
            'Category',
            'Book Name',
            'Book Rate',
            'Book Price(£)',
            'Book Availability',
            'Book Url'
        ])

        if m == 'w':
            writer.writeheader()
            
        for book_name, book_price, book_rate, ava_stock,book_url in zip(names , rates , price , stock , books_url): 
        
            writer.writerow({
                'Category' : cat,
                'Book Name' : book_name,
                'Book Price(£)' : book_price,
                'Book Rate' : book_rate,
                'Book Availability' : ava_stock,
                'Book Url' : book_url
                })
        
    
    
    
    

In [60]:
for cat , cat_url in cats.items():

    # scrape for first page for each category
    names , rates , price , stock , books_url , next_pg = scrape(cat_url)
    print(cat_url)
    StoreData(cat , names , rates , price , stock , books_url)
    
    while next_pg != '':
        
        new_url = cat_url.replace('index.html' , next_pg)
        print(new_url)
        names , rates , price , stock , books_url , next_pg = scrape(new_url)
        StoreData(cat , names , rates , price , stock , books_url)
        
        
    print('*'*30)

https://books.toscrape.com/catalogue/category/books/travel_2/index.html
******************************
https://books.toscrape.com/catalogue/category/books/mystery_3/index.html
https://books.toscrape.com/catalogue/category/books/mystery_3/page-2.html
******************************
https://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html
https://books.toscrape.com/catalogue/category/books/historical-fiction_4/page-2.html
******************************
https://books.toscrape.com/catalogue/category/books/sequential-art_5/index.html
https://books.toscrape.com/catalogue/category/books/sequential-art_5/page-2.html
https://books.toscrape.com/catalogue/category/books/sequential-art_5/page-3.html
https://books.toscrape.com/catalogue/category/books/sequential-art_5/page-4.html
******************************
https://books.toscrape.com/catalogue/category/books/classics_6/index.html
******************************
https://books.toscrape.com/catalogue/category/books/philosop

In [44]:
names 

["It's Only the Himalayas",
 'Full Moon over Noah’s Ark: An Odyssey to Mount Ararat and Beyond',
 'See America: A Celebration of Our National Parks & Treasured Sites',
 'Vagabonding: An Uncommon Guide to the Art of Long-Term World Travel',
 'Under the Tuscan Sun',
 'A Summer In Europe',
 'The Great Railway Bazaar',
 'A Year in Provence (Provence #1)',
 'The Road to Little Dribbling: Adventures of an American in Britain (Notes From a Small Island #2)',
 'Neither Here nor There: Travels in Europe',
 '1,000 Places to See Before You Die']

In [10]:
section = soup.find('section')

In [12]:
ol = section.find('ol')

In [14]:
lis = ol.find_all('li')

In [21]:
book_name = lis[0].find('div' , attrs = {'class' : 'image_container'}).find('a').find('img')['alt']
book_name

"It's Only the Himalayas"

In [28]:
book_rate = lis[0].find('p' , attrs = {'class' : 'star-rating'})['class'][1]
book_rate

'Two'

In [34]:
book_price = float(lis[0].find('p' , attrs = {'class' : 'price_color'}).text[1:])
book_price

45.17

In [41]:
ava_stock = lis[0].find('p' , attrs = {'class' : 'instock availability'}).text.strip()
ava_stock

'In stock'

In [48]:
print('https://books.toscrape.com/catalogue/' + lis[0].find('h3').find('a')['href'].replace('../' , ''))

https://books.toscrape.com/catalogue/its-only-the-himalayas_981/index.html


In [49]:
for li in lis:
    
    book_name = li.find('div' , attrs = {'class' : 'image_container'}).find('a').find('img')['alt']
    print(book_name)
    
    book_rate = li.find('p' , attrs = {'class' : 'star-rating'})['class'][1]
    print(book_rate)
    
    book_price = float(li.find('p' , attrs = {'class' : 'price_color'}).text[1:])
    print(book_price)
    
    ava_stock = li.find('p' , attrs = {'class' : 'instock availability'}).text.strip()
    print(ava_stock)
    
    book_url = 'https://books.toscrape.com/catalogue/' + li.find('h3').find('a')['href'].replace('../' , '')
    print(book_url)
    
    
    print('*'*30)
    

It's Only the Himalayas
Two
45.17
In stock
https://books.toscrape.com/catalogue/its-only-the-himalayas_981/index.html
******************************
Full Moon over Noah’s Ark: An Odyssey to Mount Ararat and Beyond
Four
49.43
In stock
https://books.toscrape.com/catalogue/full-moon-over-noahs-ark-an-odyssey-to-mount-ararat-and-beyond_811/index.html
******************************
See America: A Celebration of Our National Parks & Treasured Sites
Three
48.87
In stock
https://books.toscrape.com/catalogue/see-america-a-celebration-of-our-national-parks-treasured-sites_732/index.html
******************************
Vagabonding: An Uncommon Guide to the Art of Long-Term World Travel
Two
36.94
In stock
https://books.toscrape.com/catalogue/vagabonding-an-uncommon-guide-to-the-art-of-long-term-world-travel_552/index.html
******************************
Under the Tuscan Sun
Three
37.33
In stock
https://books.toscrape.com/catalogue/under-the-tuscan-sun_504/index.html
******************************
A 

In [53]:
import time 

In [56]:
with open('Books Data.csv' , 'w' , newline='' , encoding='utf-8') as f :
    
    
    writer = csv.DictWriter(f , fieldnames=[
        'Book Name',
        'Book Rate',
        'Book Price(£)',
        'Book Availability',
        'Book Url'
    ])

    
    writer.writeheader()
    
    for pgn in range(1 , 51):

        url = f'https://books.toscrape.com/catalogue/category/books_1/page-{pgn}.html'


        req = requests.get(url)
        con = req.content

        soup = BeautifulSoup(con , 'html.parser')

        section = soup.find('section')

        ol = section.find('ol')

        lis = ol.find_all('li')


        for li in lis:

            book_name = li.find('div' , attrs = {'class' : 'image_container'}).find('a').find('img')['alt']
    #         print(book_name)

            book_rate = li.find('p' , attrs = {'class' : 'star-rating'})['class'][1]
    #         print(book_rate)

            book_price = float(li.find('p' , attrs = {'class' : 'price_color'}).text[1:])
    #         print(book_price)

            ava_stock = li.find('p' , attrs = {'class' : 'instock availability'}).text.strip()
    #         print(ava_stock)

            book_url = 'https://books.toscrape.com/catalogue/' + li.find('h3').find('a')['href'].replace('../' , '')
    #         print(book_url)
    
            writer.writerow({
                'Book Name' : book_name,
                'Book Price(£)' : book_price,
                'Book Rate' : book_rate,
                'Book Availability' : ava_stock,
                'Book Url' : book_url
            })
    #         print('*'*30)


        print(f'{pgn}  Done!!')
        time.sleep(1)


1  Done!!
2  Done!!
3  Done!!
4  Done!!
5  Done!!
6  Done!!
7  Done!!
8  Done!!
9  Done!!
10  Done!!
11  Done!!
12  Done!!
13  Done!!
14  Done!!
15  Done!!
16  Done!!
17  Done!!
18  Done!!
19  Done!!
20  Done!!
21  Done!!
22  Done!!
23  Done!!
24  Done!!
25  Done!!
26  Done!!
27  Done!!
28  Done!!
29  Done!!
30  Done!!
31  Done!!
32  Done!!
33  Done!!
34  Done!!
35  Done!!
36  Done!!
37  Done!!
38  Done!!
39  Done!!
40  Done!!
41  Done!!
42  Done!!
43  Done!!
44  Done!!
45  Done!!
46  Done!!
47  Done!!
48  Done!!
49  Done!!
50  Done!!
